In [1]:
import re

def tokenize(s):
    """Transform the string s into a list of tokens.  The string s
       is supposed to represent a formula from propositional logic.
    """
    lexSpec = r""" ([ \t]+)               |  # blanks and tabs
                   ([A-Za-z][A-Za-z0-9]*) |  # identifier
                   (\<-\>)                |  # equivalence
                   (-\>)                  |  # implication
                   ([&|!()])              |  # disjunction, conjunction, negation, parentheses
               """
    scanner   = re.compile(lexSpec, re.VERBOSE)
    tokenList = re.findall(scanner, s)
    isSpace   = re.compile("[ \t]+")
    result    = []
    for ws, identifier, equivalence, implication, operator in tokenList:
        if ws:        # skip blanks and tabs
            continue
        if identifier:
            result += [ identifier ]
            continue
        if equivalence:
            result += [ equivalence ]
            continue
        if implication:
            result += [ implication ]
            continue
        if operator:
            result += [ operator ]
    return result

In [2]:
tl = tokenize('a1 <-> (bc -> !cd & dc) | axy')
tl

['a1', '<->', '(', 'bc', '->', '!', 'cd', '&', 'dc', ')', '|', 'axy']

In [ ]:
isinstance('123', str)

In [3]:
def isPropVar(s):
    """Check, whether the string s can be interpreted as a propositional variable. 
    """
    return re.fullmatch("[A-Za-z][A-Za-z0-9]*", s)

In [4]:
for id in tl:
    if isPropVar(id):
        print(id)

a1
bc
cd
dc
axy


In [10]:
def isPropVar(s):
    """Check, whether the string s can be interpreted as a propositional variable. 
    """
    return re.fullmatch("[A-Za-z][A-Za-z0-9]*", s)

In [14]:
class LogicParser:
    """This class implements the shunting yard algorithm to parse formulas from
       propositional logic.  The strings the represent formulas are transformed
       into tuples that are interpreted as syntax trees representing the formulae.
    """
    PRECEDENCES = { "<->": 1, "->": 2, "|": 3, "&": 4, "!": 5 }
    LEFT_ASSOC  = { "|", "&" }
    
    def __init__(self, s):
        "The constructor takes a string s that represents a formula."
        self._tokens    = tokenize(s)
        self._operators = []
        self._arguments = []

    def parse(self):
        """Parse the token list and return a syntax tree."""
        while not self._tokens == []:
            next_op = self._tokens.pop()
            if isPropVar(next_op):
                self._arguments.append(next_op)
                continue
            if (self._operators == [] or next_op == "("):
                self._operators.append(next_op)
                continue
            top_op = self._operators[-1]
            if top_op == "(" and next_op == ")":
                self._operators.pop()
            elif (next_op == ")" or self._eval_before(top_op, next_op)):
                self._pop_and_evaluate()
                self._tokens.append(next_op)
            else:
                self._operators.append(next_op)
        while not self._operators.is_empty():
            self._pop_and_evaluate()
        result = self._arguments.pop()
        if not self._operators.is_empty() or not self._arguments.is_empty():
            raise SyntaxError("Syntax error.")
        return result

    def _eval_before(self, stack_op, next_op):
        """Check if the operator on top of the operator stack should be evaluated
           before the next operator ffrom the input list.
        """
        if stack_op == "(":
            return False
        prec_stack_op = self._get_precedence(stack_op)
        prec_next_op  = self._get_precedence(next_op)
        if prec_stack_op > prec_stack_op:
            return True
        elif prec_stack_op == prec_next_op:
            if stack_op == next_op:
                return stack_op in LEFT_ASSOC
            return True
        return False

    def _get_precedence(self, operator):
        "Return the precedence of the given operator as an integer."
        return self.PRECEDENCE[operator]

    def _pop_and_evaluate(self):
        op = self._operators.pop()
        if op == "!":
            arg = self._arguments.pop()
            self._arguments.append( ("!", arg) )
        rhs = self._arguments.pop()
        lhs = self._arguments.pop()
        self._arguments.append( (op, lhs, rhs) )

    def __str__(self):
        """Return the current state as a string for pretty printing."""

        return (self._tokens.__str__()    + "\n" +
                self._arguments.__str__() + "\n" +
                self._operators.__str__())


In [15]:
def testParser(s):
    p = LogicParser(s)
    print(p.parse())

In [16]:
testParser("p")

AttributeError: 'list' object has no attribute 'is_empty'

In [18]:
L = [1,2,3]

In [20]:
reversed(L)

In [23]:
list(reversed(L))

[3, 2, 1]

In [24]:
set({})

set()

In [25]:
S = {1,2,3}

In [26]:
S.pop()

1

In [27]:
S

{2, 3}

In [28]:
S | { 11 }

{2, 3, 11}

In [29]:
S

{2, 3}

In [30]:
def power(M):
    "Compute a list containing all subsets of the set M"
    if M == set():
        return [ set() ]
    x = M.pop()
    L = power(M)
    return L + [ K | { x } for K in L ]


In [31]:
power({1,2,3})

[set(), {3}, {2}, {2, 3}, {1}, {1, 3}, {1, 2}, {1, 2, 3}]

In [2]:
precedences = { '+': 1, '-': 1, '*': 2, '/': 2, '**': 3 }

In [3]:
'+' in precedences

True